#### This notebook cleans the downloaded stock prices from Yahoo Finance 

In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'DOC',
       'ESAB', 'FCFS', 'GEN', 'IBKR', 'JJC', 'NSTG', 'PFG', 'SBNY', 'SRPT',
       'SSB', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ABM', 'ACM', 'ACN', 'AGCO', 'ALE', 'ALRM', 'AMCR', 'AMED', 'AMG',
       'AMGN',
       ...
       'VRSK', 'VSAT', 'WDAY', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=254)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'BJ', 'BRBR', 'BVH', 'CARR', 'CDAY', 'CEG',
       'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG', 'DOW', 'DT', 'DTM',
       'ESAB', 'ETRN', 'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO',
       'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'NARI', 'NVST', 'NVT', 'OGN', 'OTIS',
       'PDD', 'PGNY', 'RIVN', 'SGOV', 'SHC', 'SITM', 'SRC', 'SWAV', 'VNT',
       'VSCO', 'WH', 'YETI', 'ZM'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 311


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2024-03-08,120,"[30, 60, 120]",[],[],[],[],[],[],[],[],"['GBTC', 'NVDA', 'SHV', 'VST']","['CLS', 'EME', 'ETH-USD', 'FATE', 'FTSM', 'GE'...","['BTC-USD', 'CELH', 'META', 'NTRA']",['WING'],"['ANF', 'DELL', 'EVBG', 'STRL']",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],[],[],"['GBTC', 'NVDA', 'SHV', 'VST']","['CLS', 'EME', 'ETH-USD', 'FATE', 'FTSM', 'GE'...","['BTC-USD', 'CELH', 'META', 'NTRA']",['WING'],"['ANF', 'DELL', 'EVBG', 'STRL']",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 57240
unique symbol count from model picks: 1006


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['ALE', 'ARCH', 'ATKR', 'AZPN', 'CHT', 'CMG', 'CVBF', 'CVLT', 'DVA', 'HOLI', 'HY', 'LOB', 'MLKN', 'MLR', 'NSTG', 'OSIS', 'PKX', 'PTMN', 'RRC', 'SFBS', 'SHYF', 'SRPT', 'TRGP', 'TSEM', 'TX', 'VNO', 'WDAY', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 311
len(common_symbols): 28
len(symbols_drop): 283


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 283


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      23
symbols with same volume:   254
symbols with no close:       51

symbols total before drop:                                        1,546
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   283

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,263    1,500
df Close (df_c) after dropped symbols:      1,263    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2024-03-04  139.149994  143.490005  138.809998  142.860001  2169200.0   
2024-03-05  143.000000  146.399994  143.000000  144.350006  2622500.0   
2024-03-06  145.000000  147.440002  144.750000  146.220001  2357900.0   
2024-03-07  147.089996  149.639999  147.089996  149.309998  1942900.0   
2024-03-08  149.639999  151.580002  147.850006  147.869995  1588300.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2024-03-04  27.500000  27.549999  27.040001  27.480000  3954700.0  ...   
2024-03-05  27.049999  27.209999  26.570000  26.910000  5704800.0  ...   
2024-03-06  27.670000  28.725000  26.900000  28.219999  6335600.0  ...   
2024-03-07  28.860001  30.100000  28.860001  29.870001  7740500.0  ...   
2024-03-08  30.379999  30.639999  29.270000  29.850000  6536400.0  ...   

                 ZUMZ                                               ZWS  \
                 Open       High        Low  Close    Volume       Open   
Date                                                                      
2024-03-04  17.520000  17.520000  16.910000  16.91  129500.0  32.040001   
2024-03-05  16.719999  16.990000  16.389999  16.42  182900.0  31.950001   
2024-03-06  16.540001  16.559999  16.040001  16.32  193200.0  31.600000   
2024-03-07  16.500000  16.590000  16.070000  16.10  146200.0  31.570000   
2024-03-08  16.340000  16.680000  16.020000  16.18  114400.0  32.349998   

                                                       
                 High        Low      Close    Volume  
Date                                                   
2024-03-04  32.509998  32.029999  32.060001  681100.0  
2024-03-05  32.099998  31.270000  31.350000  521600.0  
2024-03-06  31.760000  31.180000  31.400000  740700.0  
2024-03-07  32.139999  31.549999  32.029999  643000.0  
2024-03-08  32.529999  31.840000  31.969999  841400.0  

[5 rows x 6315 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,...,ZBH,ZBRA,ZD,ZG,ZION,ZS,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-04,142.860001,27.480000,14.81,69.660004,175.100006,32.930000,177.050003,12.50,120.040001,87.139999,...,125.510002,291.649994,68.879997,55.369999,39.169998,214.220001,20.450001,187.860001,16.91,32.060001
2024-03-05,144.350006,26.910000,14.67,72.660004,170.119995,33.610001,179.160004,12.75,118.480003,87.389999,...,124.779999,284.540009,65.550003,52.400002,40.950001,206.360001,20.049999,185.229996,16.42,31.350000
2024-03-06,146.220001,28.219999,14.88,71.900002,169.119995,33.840000,181.059998,12.80,119.339996,88.820000,...,125.830002,287.269989,66.879997,54.029999,41.299999,202.419998,20.440001,183.529999,16.32,31.400000
2024-03-07,149.309998,29.870001,14.90,73.680000,169.000000,34.000000,180.570007,12.91,120.919998,87.690002,...,127.360001,288.589996,67.150002,54.060001,41.520000,201.270004,20.070000,181.509995,16.10,32.029999
2024-03-08,147.869995,29.850000,14.68,73.080002,170.729996,33.830002,178.850006,12.86,120.959999,87.610001,...,126.739998,282.589996,66.769997,56.150002,42.240002,198.690002,20.139999,182.160004,16.18,31.969999
